In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [37]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
import joblib
import nltk
from nltk.corpus import stopwords
import string
from sklearn.linear_model import LogisticRegression


In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
dataset_path = "/kaggle/input/articles/articles.csv"
df = pd.read_csv(dataset_path, encoding = 'latin-1')

In [11]:
df

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours dOlivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive
...,...,...,...,...,...,...,...,...
4300,a40e5017-0a38-4d06-bcbe-616b73456c94,"Chinook Catches Army Flirting With Younger, Th...",https://www.duffelblog.com/wp-content/uploads/...,Duffel Blog,Long rumored tensions came to a head as the CH...,<p>Long rumored tensions came to a head as the...,Military,NaN
4301,782ba519-bcb0-4ef1-873d-713a18b04576,Lufthansa Aviation Selects Reiser Simulation H...,NaN,Vertical,<p>In the course of upcoming investments in ne...,<p>In the course of upcoming investments in ne...,Commercial,Positive
4302,deb31e5d-15c0-4c1d-843c-ce02e9081746,This Bell Flight Drone Wont Be Delivering Pizza,NaN,"WFAA-TV ABC (Dallas, TX)","<p>At Bell Flight in Fort Worth, engineers are...","<p>At Bell Flight in Fort Worth, engineers are...",Military,Positive
4303,f7125b1d-a687-469b-a799-c8cb4443b1d1,Blade Offers New York Airport Transfers for $1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,Bloomberg,<p>Getting to this price point took about five...,<p>Getting to this price point took about five...,Commercial,Positive


In [14]:
df.head

<bound method NDFrame.head of                                         Id  \
0     d6995462-5e87-453b-b64d-e9f1df6e94d2   
1     8b05e939-a89e-4548-b92b-013822e8ee7d   
2     69fcd400-bceb-4255-8277-619f2d68ac0b   
3     17943578-c11b-414b-b3f5-063d3a93157b   
4     f33c7b11-5f77-4a98-bb2e-d36689042aea   
...                                    ...   
4300  a40e5017-0a38-4d06-bcbe-616b73456c94   
4301  782ba519-bcb0-4ef1-873d-713a18b04576   
4302  deb31e5d-15c0-4c1d-843c-ce02e9081746   
4303  f7125b1d-a687-469b-a799-c8cb4443b1d1   
4304  c0d27375-9fc2-43dc-b709-26a2c48c462b   

                                                Heading  \
0     A Puzzling Maneuver, Then Freefall: NTSB Repor...   
1     Bells Nexus Air Taxi Concept Rings Changes Fo...   
2                   Bell Helicopter Show Air Taxi Nexus   
3     BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...   
4                Les premiers retours dOlivier Ezratty   
...                                                 ...   
4300

In [15]:
df.shape

(4305, 8)

In [16]:
df = df.dropna()

In [17]:
df = df.drop_duplicates()

In [18]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

In [27]:
df['Full_Article'] = df['Heading'].apply(preprocess_text)

X = df['Full_Article']
y = df['Article_Type']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [35]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=cf35a4eb5c6ee8dc54a5ab4fbc57a483fd266d8ef726c24882dfa5ee8c4b15f6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [36]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

X_train_embeddings = model.encode(X_train.tolist(), convert_to_tensor=True)
X_test_embeddings = model.encode(X_test.tolist(), convert_to_tensor=True)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [43]:

tfidf_vectorizer = TfidfVectorizer(max_features=5000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

from sklearn.linear_model import LogisticRegression

logistic_classifier = LogisticRegression(max_iter=1000)
logistic_classifier.fit(X_train_tfidf, y_train_encoded)

y_pred = logistic_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_encoded, y_pred))


new_text = ["Long rumored tensions came to"]
new_text_tfidf = tfidf_vectorizer.transform(new_text)
prediction = logistic_classifier.predict(new_text_tfidf)
predicted_class = le.inverse_transform(prediction) 
print("Predicted class for new text:", predicted_class)

Accuracy: 0.9228187919463087
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       186
           1       0.96      0.85      0.90       109
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1

    accuracy                           0.92       298
   macro avg       0.47      0.46      0.46       298
weighted avg       0.92      0.92      0.92       298

Predicted class for new text: ['Commercial']


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
import pickle

with open('logistic_text_classifier.pkl', 'wb') as file:
    pickle.dump(logistic_classifier, file)


In [45]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # You can adjust these values
    'penalty': ['l1', 'l2'],
}


In [46]:
from sklearn.model_selection import GridSearchCV


logistic_classifier = LogisticRegression(max_iter=1000)


grid_search = GridSearchCV(estimator=logistic_classifier, param_grid=param_grid, scoring='accuracy', cv=5)

grid_search.fit(X_train_tfidf, y_train_encoded)


best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Best Model - Accuracy:", accuracy)


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Best Model - Accuracy: 0.9194630872483222


In [48]:
import pickle
with open('best_logistic_text_classifier.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [49]:
with open('best_logistic_text_classifier.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [51]:
import requests
from bs4 import BeautifulSoup

def extract_heading_and_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 

        soup = BeautifulSoup(response.text, 'html.parser')

        heading = soup.find('h1').text 

        article_text = ' '.join([p.text for p in soup.find_all('p')]) 

        return heading, article_text

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None

article_url = 'https://www.airmedandrescue.com/story/113231/airbus-expands-north-american-presence'
heading, full_article = extract_heading_and_article(article_url)

if heading and full_article:
    print("Heading:", heading)
    print("Full Article:")
    print(full_article)
else:
    print("Failed to extract heading and/or full article.")


Heading: 
Airbus expands North American presence

Full Article:
Airbus Helicopters has expanded its presence in the North American emergency medical services (EMS) market with the sale of six helicopters for two customers at the recent 2018 Air Medical Transport Conference. Three H125s and two H130s are being added to REACH Air Medical Services’ current fleet and are slated for delivery this year. It is hoped the additions will enable REACH to expand its services area, in addition to modernising its fleet. Another new customer – Medical Air Rescue Company (MARC) –signed for its first Airbus helicopter, the H130. This took place as part of MARC’s long-term plan to modernise its fleet and expand its operations into Wyoming, where the aircraft will be based. © Voyageur Publishing & Events 2019
